In [2]:
import pandas as pd

In [17]:
dfm = pd.read_csv('C:/Users/D.Elistratov/Documents/Презентации лекций/Неделя 6/ПИТОН/Marketing_pilot.csv')
df = pd.read_csv('C:/Users/D.Elistratov/Documents/Презентации лекций/Неделя 6/ПИТОН/SKYBANK.csv')

In [18]:
dfm['ad_date'] = pd.to_datetime(dfm['ad_date'])
dfm['purchase_date'] = pd.to_datetime(dfm['purchase_date'])
dfm['time_left'] = pd.to_datetime(dfm['time_left'])

In [19]:
dfm['flag_30'] = (dfm['ad_date'] + pd.Timedelta(days=30) > dfm['purchase_date']).astype(int)
dfm['flag_60'] = (dfm['ad_date'] + pd.Timedelta(days=60) > dfm['purchase_date']).astype(int)
dfm['flag_90'] = (dfm['ad_date'] + pd.Timedelta(days=90) > dfm['purchase_date']).astype(int)

In [20]:
'''

select promo_type, id_group, count(user_id) as cnt_all, sum(flag_30) as cnt_30, sum(flag_60) as cnt_60, sum(flag_90) as cnt_90
from dfm
where promo_type<>'Gold Bundle'
group by promo_type, id_group

'''

dfm_gr = dfm[dfm['promo_type']!='Gold Bundle'].groupby(['promo_type','id_group']).agg({'user_id':'count'
                                                , 'flag_30': 'sum'
                                                , 'flag_60': 'sum'
                                                , 'flag_90': 'sum'
                                                    }).reset_index().rename(columns={'user_id':'cnt_all'
                                                                                   , 'flag_30': 'cnt_30'
                                                                                   , 'flag_60': 'cnt_60'
                                                                                   , 'flag_90': 'cnt_90'
                                                                                     }  
                                                                            )

"\n\nselect promo_type, id_group, count(user_id) as cnt_all, sum(flag_30) as cnt_30, sum(flag_60) as cnt_60, sum(flag_90) as cnt_90\nfrom dfm\nwhere promo_type<>'Gold Bundle'\ngroup by promo_type, id_group\n\n"

In [21]:
dfm_gr['ratio_30'] = dfm_gr['cnt_30'] / dfm_gr['cnt_all']
dfm_gr['ratio_60'] = dfm_gr['cnt_60'] / dfm_gr['cnt_all']
dfm_gr['ratio_90'] = dfm_gr['cnt_90'] / dfm_gr['cnt_all']

In [22]:
'''
select sum(amt_loan)
        , credit_type
from df
where gender is not null
group by credit_type
'''

df_gr_cnt_crtype = df[df['gender'].notnull()].groupby('credit_type').sum()['amt_loan'].reset_index()

In [23]:
'''
select count(id_client)
        , credit_type
        , cl_segm
        , gender
from df
where gender is not null
group by credit_type
        , cl_segm
        , gender
'''

df_gr_3 = df[(df['gender'].notnull())].groupby(['credit_type','cl_segm','gender']).count()['id_client'].reset_index()

In [25]:
'''
select sum(amt_loan)
        , credit_type
        , cl_segm
from df
where amt_loan > 100000 and is_active = 1
group by credit_type
        , cl_segm
'''

df_gr_4 = df[(df['amt_loan'] > 100000) & (df['is_active'] == 1)].groupby(['credit_type','cl_segm']).sum()[['amt_loan','age']].reset_index()

In [26]:
'''
select sum(amt_loan) as sum_credit
        , count(*) as cnt_all
        , credit_type
        , gender
from df
where amt_loan > 10000 and cellphone is not null
group by credit_type
        , gender
'''

df_gr_5 = df[(df['amt_loan'] > 10000) 
           & (df['cellphone'].notnull())].groupby(['credit_type'
                                                  ,'gender']).agg({"amt_loan":'sum'
                                                                            , 'id_client':'count'
                                                                  }).reset_index().rename(columns = {'amt_loan':'sum_credit'
                                                                                                     , 'id_client':'cnt_all'
                                                                                                     , 'gender': 'genderrr'
                                                                                                    })

In [27]:
df['date_loan'] = pd.to_datetime(df['date_loan'])

df['date_loan_offset'] = df['date_loan'] + pd.DateOffset(months=1) + pd.DateOffset(days=2) + pd.DateOffset(hours=3)


In [29]:
df['delta'] = df['date_loan_offset'] - df['date_loan'] 
df['delta_days'] = df['delta'].dt.components['days']

In [30]:
df['percent_delta_days'] = df['delta_days'] / df['delta_days'].sum()*100